# Image Similarity Estimation using a Siamese Network
The goal: given a database of anime images and an image of the user's face, return the top 5 anime characters that most closely resemble the user.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting and graphics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import random

import keras
from keras import applications
from keras.applications import resnet
from keras import ops, layers
from keras import optimizers, metrics

import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

TARGET_SHAPE=(256, 256)
os.environ["KERAS_BACKEND"] = "tensorflow"
autotune = tf.data.AUTOTUNE

As a placeholder dataset, we use the **Totally Looks Like** dataset to test the capabilities of the Siamese Network. Once the **CycleGAN** is able to amass a repository of real/anime image-pairs, we can train **ISSN** on our custom dataset.

In [ ]:
def preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, TARGET_SHAPE)
    return image

In [ ]:
def load_dataset(path):
    image_paths = tf.data.Dataset.list_files(
        os.path.join(path, '*'),
        shuffle=False
    )
    dataset = image_paths.map(
        preprocess_image,
        num_parallel_calls=autotune
    )
    return dataset

In [ ]:
AnchorImages = load_dataset('/kaggle/input/totally-looks-like-dataset/left/left')
PositiveImages = load_dataset('/kaggle/input/totally-looks-like-dataset/right/right')

In [ ]:
NegativeImages = tf.data.experimental.sample_from_datasets([AnchorImages, PositiveImages])

In [ ]:
ISSNDataset = tf.data.Dataset.zip((
    AnchorImages,
    PositiveImages,
    NegativeImages
)).shuffle(buffer_size=1024)

In [ ]:
SPLIT = round(len(AnchorImages) * 0.8)
ISSNTrain = ISSNDataset.take(SPLIT).batch(32, drop_remainder=False).prefetch(autotune)
ISSNVal = ISSNDataset.skip(SPLIT).batch(32, drop_remainder=False).prefetch(autotune)

We can look at a few examples of the triplets. The Anchor and Positive images should match, while the Negative image is *always*different.

In [ ]:
def visualize(anchor, positive, negative):
    def show(ax, image):
        ax.imshow(image)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    fig = plt.figure(figsize=(9, 9))
    axs = fig.subplots(3, 3)
    for i in range(3):
        show(axs[i, 0], anchor[i])
        show(axs[i, 1], positive[i])
        show(axs[i, 2], negative[i])

In [ ]:
batch = next(
    iter(
        ISSNTrain.take(1).as_numpy_iterator()
    )
)  # Get one batch
visualize(*batch)  # Unpack the batch

## Create a Siamese Network from the ResNET 50 Model

In [ ]:
ResNET50 = resnet.ResNet50(
    weights = "imagenet",
    input_shape = TARGET_SHAPE + (3,),
    include_top = False
)

The Siamese Network will generate embeddings for each of the images of the triplet using a **ResNet50** model pretrained on ImageNet. It will connect to a few Dense layers so we can learn to separate these embeddings.

We will freeze the weights of all the layers of the model up until the layer `conv5_block1_out`to avoid changing the weights that the model has already learned (that would be bad!). Thus, we are going to leave the bottom few layers trainable so that we can fine-tune their weights during training.

In [ ]:
flatten = layers.Flatten()(ResNET50.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

Embedder = keras.Model(ResNET50.input, output, name="ResNET50_Embedding")

In [ ]:
trainable = False
for layer in ResNET50.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

We also need a **Custom Layer** to compute the distance between the anchor and the positive embedding, as well as the distance between the anchor and the negative embedding.

In [ ]:
class APNDistance(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = ops.sum(tf.square(anchor - positive), -1)
        an_distance = ops.sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)

In [ ]:
anchor_input = layers.Input(name="anchor", shape=TARGET_SHAPE + (3,))
positive_input = layers.Input(name="positive", shape=TARGET_SHAPE + (3,))
negative_input = layers.Input(name="negative", shape=TARGET_SHAPE + (3,))

In [ ]:
Distance = APNDistance()(
    Embedder(resnet.preprocess_input(anchor_input)),
    Embedder(resnet.preprocess_input(positive_input)),
    Embedder(resnet.preprocess_input(negative_input))
)

In [ ]:
SiameseNetworkModel = keras.Model(
    inputs = [anchor_input, positive_input, negative_input],
    outputs = Distance,
    name = "SiameseNetworkModel"
)

In [ ]:
SiameseNetworkModel.summary()

## Train the Model 
Implement a model with custom training loop so we can compute the triplet loss using the three embeddings produced by the Siamese network.

In [ ]:
class SiameseImageSimilarityModel(keras.Model):
    def __init__(self, actual_network, margin=.5):
        super().__init__()
        self.siamese_network = actual_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        ap_distance, an_distance = self.siamese_network(data)
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0) #can't have negative loss
        return loss

    @property
    def metrics(self):
        return [self.loss_tracker]

In [ ]:
siamese_model = SiameseImageSimilarityModel(SiameseNetworkModel)
siamese_model.compile(optimizer=optimizers.Adam(0.0001))
history = siamese_model.fit(ISSNTrain, epochs=15, validation_data=ISSNVal)

In [ ]:
sample = next(iter(ISSNTrain))
visualize(*sample)

In [ ]:
anchor, positive, negative = sample
anchor_embedding, positive_embedding, negative_embedding = (
    Embedder(resnet.preprocess_input(anchor)),
    Embedder(resnet.preprocess_input(positive)),
    Embedder(resnet.preprocess_input(negative)),
)

In [ ]:
cosine_similarity = metrics.CosineSimilarity()

positive_similarity = cosine_similarity(anchor_embedding, positive_embedding)
print("Positive similarity:", positive_similarity.numpy())

negative_similarity = cosine_similarity(anchor_embedding, negative_embedding)
print("Negative similarity", negative_similarity.numpy())

In [ ]:
# Extract data
history_dict = history.history
loss = history_dict["loss"]
val_loss = history_dict["val_loss"]
accuracy = history_dict.get("accuracy")  # Works for classification models
val_accuracy = history_dict.get("val_accuracy")

epochs = range(1, len(loss) + 1)

# Plot Loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, "bo-", label="Training Loss")
plt.plot(epochs, val_loss, "r^-", label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Training & Validation Loss")

# Plot Accuracy (if available)
if accuracy:
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, "bo-", label="Training Accuracy")
    plt.plot(epochs, val_accuracy, "r^-", label="Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title("Training & Validation Accuracy")

plt.show()


In [ ]:
results = siamese_model.evaluate(ISSNVal)

In [ ]:
print(results)